In [1]:
print("JOHN")

JOHN


In [20]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [21]:
PINECONE_API_KEY="c632ff13-5cb2-4778-a5f2-c47fc27d77ce"
PINECONE_API_ENV="us-east-1"

In [22]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
   
    return documents

In [23]:
extracted_data = load_pdf("data/")

could not convert string to float: '0.000-4461942' : FloatObject (b'0.000-4461942') invalid; use 0.0 instead
could not convert string to float: '0.000-4461942' : FloatObject (b'0.000-4461942') invalid; use 0.0 instead
could not convert string to float: '0.000-4461942' : FloatObject (b'0.000-4461942') invalid; use 0.0 instead
could not convert string to float: '0.000-4461942' : FloatObject (b'0.000-4461942') invalid; use 0.0 instead
could not convert string to float: '0.000-4461942' : FloatObject (b'0.000-4461942') invalid; use 0.0 instead
could not convert string to float: '0.00-26634705' : FloatObject (b'0.00-26634705') invalid; use 0.0 instead
could not convert string to float: '0.000-4461942' : FloatObject (b'0.000-4461942') invalid; use 0.0 instead
could not convert string to float: '0.000-4461942' : FloatObject (b'0.000-4461942') invalid; use 0.0 instead
could not convert string to float: '0.000-4461942' : FloatObject (b'0.000-4461942') invalid; use 0.0 instead
could not convert s

In [24]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunks_size = 800, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks 

In [26]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

TypeError: __init__() got an unexpected keyword argument 'chunks_size'

In [13]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()

In [15]:
query_result = embeddings.embed_query("Hello World")
print("Length",len(query_result))

Length 384


In [28]:
pinecone.init(api_key="PINECONE_API_KEY",
              environment="PINECONE_API_ENV")

index_name="medical-chatbot"

docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [29]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)

query = "What are allergies?"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

NameError: name 'index_name' is not defined

In [30]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [31]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [33]:
llm=CTransformers(model="C:\Users\JOHN\Documents\medicalchatbot\model\model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3096109834.py, line 1)

In [34]:

qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

NameError: name 'llm' is not defined

In [35]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])